In [2]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogs import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-07-19 21:48:28.844546


In [4]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Warning! Local Discog Directory /Users/tgadfort/Documents/music/Discog is not Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/special exists
/Volumes/Music/Discog/artist-special exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


# Collections

In [45]:
%load_ext autoreload
%autoreload
col = collections(disc)
#col.parseCollections(force=True)
col.createCollectionDBs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sat Jul 20, 2019 14:40:25 for Creating Initial Artist Database
/Volumes/Music/Discog/collections-db/collections-0.p 	18482
/Volumes/Music/Discog/collections-db/collections-1.p 	36837
/Volumes/Music/Discog/collections-db/collections-10.p 	279076
/Volumes/Music/Discog/collections-db/collections-11.p 	489191
/Volumes/Music/Discog/collections-db/collections-12.p 	549490
/Volumes/Music/Discog/collections-db/collections-13.p 	588505
/Volumes/Music/Discog/collections-db/collections-14.p 	612979
/Volumes/Music/Discog/collections-db/collections-15.p 	635321
/Volumes/Music/Discog/collections-db/collections-16.p 	647227
/Volumes/Music/Discog/collections-db/collections-2.p 	657505
/Volumes/Music/Discog/collections-db/collections-3.p 	661635
/Volumes/Music/Discog/collections-db/collections-4.p 	665711
/Volumes/Music/Discog/collections-db/collections-5.p 	736477
/Volumes/Music/Discog/collections-d

# Artists

In [ ]:
%load_ext autoreload
%autoreload
arts = artists(disc)
#art.findKnownArtists(debug=True)
#art.findUnkn3wssownArtists(minVal=200, debug=True)
#art.downloadUnknownArtists(debug=True)
#arts.parseArtistFiles()
#art.searchDiscogForArtist("$uicideboy$")
arts.buildMetadata()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sat Jul 20, 2019 18:02:33 for Building Artist Metadata DB
Loading data from /Volumes/Music/Discog/db/artistNameToID.p
  --> This file is 6.1MB.
Loading /Volumes/Music/Discog/db/artistNameToID.p
Loading data from /Volumes/Music/Discog/db/artistIDToName.p
  --> This file is 6.1MB.
Loading /Volumes/Music/Discog/db/artistIDToName.p
Loading data from /Volumes/Music/Discog/db/artistRefToID.p
  --> This file is 7.2MB.
Loading /Volumes/Music/Discog/db/artistRefToID.p
Loading data from /Volumes/Music/Discog/db/artistIDToRef.p
  --> This file is 7.2MB.
Loading /Volumes/Music/Discog/db/artistIDToRef.p
Loading data from /Volumes/Music/Discog/db/artistRefToName.p
  --> This file is 8.3MB.
Loading /Volumes/Music/Discog/db/artistRefToName.p
Loading data from /Volumes/Music/Discog/db/artistNameToRef.p
  --> This file is 8.4MB.
Loading /Volumes/Music/Discog/db/artistNameToRef.p
Current Time is Sat Ju

In [20]:
n2id = getFile("/Volumes/Music/Discog/db/NameToID.p")
print(len(n2id))

1190604


In [21]:
n2id = getFile("/Volumes/Music/Discog/db/ArtistNameToID.p")
print(len(n2id))

301848


In [22]:
data = getFile("/Volumes/Music/Discog/artists-db/99-DB.p")
print(len(data))
#from collections import Counter

3000


In [16]:

cntr = Counter()
for artistID,artistData in data.items():
    media = artistData.get("Media")
    if media is None:
        continue
    for mediaName,mediaData in artistData["Media"].items():
        cntr[mediaName] += len(mediaData)
cntr.most_common()

[('Singles & EPs', 8140),
 ('Albums', 7317),
 ('Technical', 4362),
 ('Writing & Arrangement', 4057),
 ('Instruments & Performance', 3733),
 ('Production', 2394),
 ('Compilations', 1855),
 ('Visual', 1713),
 ('Miscellaneous', 988),
 ('Vocals', 882),
 ('Acting, Literary & Spoken', 484),
 ('Management', 236),
 ('Remix', 207),
 ('Videos', 188),
 ('DJ Mixes', 157),
 ('Conducting & Leading', 105),
 ('Featuring & Presenting', 65),
 ('Mixes', 56),
 ('DJ Mix', 1)]

In [53]:
albs = albums(disc)
albs.parseAlbumFiles()

Current Time is Fri Jul 19, 2019 08:19:27 for Analyzing Modval NAN
Looking for artist albums in /Volumes/Music/Discog/albums/NAN
Found 1 artist directories
  Found 2428 albums (0    ) in /Volumes/Music/Discog/albums/NAN/NAN
Saving 776 new album IDs to /Volumes/Music/Discog/albums-db/NAN-DB.p
  --> This file is 2.2MB.
Current Time is Fri Jul 19, 2019 08:20:35 for Done with Analyzing Modval NAN
Process [Done with Analyzing Modval NAN] took 1.1 minutes.
Current Time is Fri Jul 19, 2019 08:20:36 for Analyzing Modval 0
Looking for artist albums in /Volumes/Music/Discog/albums/0
Found 590 artist directories
  Found 5   albums (0    ) in /Volumes/Music/Discog/albums/0/1000
  Found 7   albums (0    ) in /Volumes/Music/Discog/albums/0/1013700
  Found 3   albums (0    ) in /Volumes/Music/Discog/albums/0/1038800
  Found 44  albums (0    ) in /Volumes/Music/Discog/albums/0/1065000
  Found 1   albums (0    ) in /Volumes/Music/Discog/albums/0/1066000
  Found 2   albums (0    ) in /Volumes/Music/Disc

  Found 15  albums (35   ) in /Volumes/Music/Discog/albums/0/25500
  Found 1   albums (35   ) in /Volumes/Music/Discog/albums/0/2551800
  Found 5   albums (35   ) in /Volumes/Music/Discog/albums/0/255500
  Found 2   albums (35   ) in /Volumes/Music/Discog/albums/0/256500
  Found 1   albums (35   ) in /Volumes/Music/Discog/albums/0/259000
  Found 7   albums (35   ) in /Volumes/Music/Discog/albums/0/2591500
  Found 3   albums (35   ) in /Volumes/Music/Discog/albums/0/260500
  Found 1   albums (35   ) in /Volumes/Music/Discog/albums/0/2605500
  Found 1   albums (35   ) in /Volumes/Music/Discog/albums/0/2608000
  Found 2   albums (35   ) in /Volumes/Music/Discog/albums/0/2625500
  Found 1   albums (35   ) in /Volumes/Music/Discog/albums/0/263000
  Found 1   albums (35   ) in /Volumes/Music/Discog/albums/0/267400
  Found 2   albums (35   ) in /Volumes/Music/Discog/albums/0/267500
  Found 3   albums (35   ) in /Volumes/Music/Discog/albums/0/268500
  Found 16  albums (35   ) in /Volumes/Music

  Found 1   albums (42   ) in /Volumes/Music/Discog/albums/0/4526500
  Found 1   albums (42   ) in /Volumes/Music/Discog/albums/0/4569000
  Found 1   albums (42   ) in /Volumes/Music/Discog/albums/0/4571500
  Found 1   albums (42   ) in /Volumes/Music/Discog/albums/0/4576000
  Found 1   albums (42   ) in /Volumes/Music/Discog/albums/0/4590000
  Found 1   albums (42   ) in /Volumes/Music/Discog/albums/0/4608000
  Found 1   albums (42   ) in /Volumes/Music/Discog/albums/0/4613000
  Found 1   albums (42   ) in /Volumes/Music/Discog/albums/0/4633000
  Found 1   albums (42   ) in /Volumes/Music/Discog/albums/0/4659500
  Found 6   albums (42   ) in /Volumes/Music/Discog/albums/0/471000
  Found 4   albums (42   ) in /Volumes/Music/Discog/albums/0/4724000
  Found 6   albums (42   ) in /Volumes/Music/Discog/albums/0/4731500
  Found 40  albums (42   ) in /Volumes/Music/Discog/albums/0/476000
  Found 5   albums (42   ) in /Volumes/Music/Discog/albums/0/4761600
  Found 1   albums (42   ) in /Volum

Current Time is Fri Jul 19, 2019 08:20:53 for Analyzing Modval 1
Looking for artist albums in /Volumes/Music/Discog/albums/1
Found 640 artist directories
  Found 1   albums (0    ) in /Volumes/Music/Discog/albums/1/1001
  Found 9   albums (0    ) in /Volumes/Music/Discog/albums/1/100501
  Found 1   albums (0    ) in /Volumes/Music/Discog/albums/1/1005501
  Found 4   albums (0    ) in /Volumes/Music/Discog/albums/1/105001
  Found 5   albums (0    ) in /Volumes/Music/Discog/albums/1/10501
  Found 3   albums (0    ) in /Volumes/Music/Discog/albums/1/1059001
  Found 1   albums (0    ) in /Volumes/Music/Discog/albums/1/1060601
  Found 6   albums (0    ) in /Volumes/Music/Discog/albums/1/11001
  Found 4   albums (0    ) in /Volumes/Music/Discog/albums/1/1131501
  Found 1   albums (0    ) in /Volumes/Music/Discog/albums/1/1140901
  Found 1   albums (0    ) in /Volumes/Music/Discog/albums/1/1145501
  Found 3   albums (0    ) in /Volumes/Music/Discog/albums/1/1148001
  Found 17  albums (0    ) 

  Found 5   albums (5    ) in /Volumes/Music/Discog/albums/1/271001
  Found 1   albums (5    ) in /Volumes/Music/Discog/albums/1/271501
  Found 4   albums (5    ) in /Volumes/Music/Discog/albums/1/272501
  Found 1   albums (5    ) in /Volumes/Music/Discog/albums/1/274401
  Found 2   albums (6    ) in /Volumes/Music/Discog/albums/1/274501
  Found 5   albums (6    ) in /Volumes/Music/Discog/albums/1/27501
  Found 4   albums (6    ) in /Volumes/Music/Discog/albums/1/2754501
  Found 4   albums (6    ) in /Volumes/Music/Discog/albums/1/276501
  Found 3   albums (6    ) in /Volumes/Music/Discog/albums/1/2774501
  Found 4   albums (6    ) in /Volumes/Music/Discog/albums/1/277501
  Found 1   albums (6    ) in /Volumes/Music/Discog/albums/1/282001
  Found 23  albums (6    ) in /Volumes/Music/Discog/albums/1/283101
  Found 6   albums (11   ) in /Volumes/Music/Discog/albums/1/2836001
  Found 1   albums (11   ) in /Volumes/Music/Discog/albums/1/2857001
  Found 4   albums (11   ) in /Volumes/Music/

...

  Found 1   albums (7    ) in /Volumes/Music/Discog/albums/66/3735066
  Found 3   albums (7    ) in /Volumes/Music/Discog/albums/66/373566
  Found 3   albums (7    ) in /Volumes/Music/Discog/albums/66/373966
  Found 2   albums (7    ) in /Volumes/Music/Discog/albums/66/3761566
  Found 3   albums (7    ) in /Volumes/Music/Discog/albums/66/376566
  Found 2   albums (7    ) in /Volumes/Music/Discog/albums/66/3780066
  Found 3   albums (7    ) in /Volumes/Music/Discog/albums/66/3782066
  Found 3   albums (7    ) in /Volumes/Music/Discog/albums/66/3806066
  Found 2   albums (7    ) in /Volumes/Music/Discog/albums/66/3812066
  Found 19  albums (7    ) in /Volumes/Music/Discog/albums/66/381566
  Found 1   albums (7    ) in /Volumes/Music/Discog/albums/66/3829066
  Found 1   albums (7    ) in /Volumes/Music/Discog/albums/66/3874066
  Found 1   albums (7    ) in /Volumes/Music/Discog/albums/66/393066
  Found 1   albums (7    ) in /Volumes/Music/Discog/albums/66/394066
  Found 8   albums (7    )

  Found 8   albums (11   ) in /Volumes/Music/Discog/albums/66/689066
  Found 3   albums (11   ) in /Volumes/Music/Discog/albums/66/692066
  Found 1   albums (11   ) in /Volumes/Music/Discog/albums/66/6926566
  Found 1   albums (11   ) in /Volumes/Music/Discog/albums/66/693066
  Found 2   albums (11   ) in /Volumes/Music/Discog/albums/66/69866
  Found 3   albums (11   ) in /Volumes/Music/Discog/albums/66/718066
  Found 2   albums (11   ) in /Volumes/Music/Discog/albums/66/731066
  Found 1   albums (11   ) in /Volumes/Music/Discog/albums/66/751066
  Found 6   albums (11   ) in /Volumes/Music/Discog/albums/66/788066
  Found 1   albums (11   ) in /Volumes/Music/Discog/albums/66/793066
  Found 5   albums (11   ) in /Volumes/Music/Discog/albums/66/79566
  Found 1   albums (11   ) in /Volumes/Music/Discog/albums/66/8066
  Found 7   albums (11   ) in /Volumes/Music/Discog/albums/66/810566
  Found 6   albums (11   ) in /Volumes/Music/Discog/albums/66/81566
  Found 1   albums (11   ) in /Volumes

  Found 146 albums (0    ) in /Volumes/Music/Discog/albums/67/208267
  Found 3   albums (22   ) in /Volumes/Music/Discog/albums/67/2094067
  Found 1   albums (22   ) in /Volumes/Music/Discog/albums/67/2096067
  Found 2   albums (22   ) in /Volumes/Music/Discog/albums/67/2100067
  Found 1   albums (22   ) in /Volumes/Music/Discog/albums/67/2109567
  Found 3   albums (22   ) in /Volumes/Music/Discog/albums/67/211067
  Found 3   albums (22   ) in /Volumes/Music/Discog/albums/67/2120067
  Found 3   albums (22   ) in /Volumes/Music/Discog/albums/67/214467
  Found 2   albums (22   ) in /Volumes/Music/Discog/albums/67/2153567
  Found 1   albums (22   ) in /Volumes/Music/Discog/albums/67/2154567
  Found 1   albums (22   ) in /Volumes/Music/Discog/albums/67/2185567
  Found 13  albums (22   ) in /Volumes/Music/Discog/albums/67/218567
  Found 2   albums (22   ) in /Volumes/Music/Discog/albums/67/2259267
  Found 3   albums (22   ) in /Volumes/Music/Discog/albums/67/230567
  Found 2   albums (22   

  Found 7   albums (23   ) in /Volumes/Music/Discog/albums/67/4067
  Found 1   albums (23   ) in /Volumes/Music/Discog/albums/67/4090067
  Found 1   albums (23   ) in /Volumes/Music/Discog/albums/67/4103567
  Found 1   albums (23   ) in /Volumes/Music/Discog/albums/67/411567
  Found 2   albums (23   ) in /Volumes/Music/Discog/albums/67/4123067
  Found 1   albums (23   ) in /Volumes/Music/Discog/albums/67/4123567
  Found 1   albums (23   ) in /Volumes/Music/Discog/albums/67/415567
  Found 1   albums (23   ) in /Volumes/Music/Discog/albums/67/418067
  Found 3   albums (23   ) in /Volumes/Music/Discog/albums/67/4311067
  Found 1   albums (23   ) in /Volumes/Music/Discog/albums/67/4314067
  Found 1   albums (23   ) in /Volumes/Music/Discog/albums/67/4439567
  Found 5   albums (23   ) in /Volumes/Music/Discog/albums/67/44567
  Found 27  albums (23   ) in /Volumes/Music/Discog/albums/67/45467
  Found 1   albums (28   ) in /Volumes/Music/Discog/albums/67/4552567
  Found 5   albums (28   ) in 

KeyboardInterrupt: 

In [41]:
from glob import glob
from fileUtils import getsize
from fsUtils import removeFile
files = glob("/Volumes/Music/Discog/albums/1/*/*.p")

In [42]:
[removeFile(y) for y in [x for x in files if getsize(x) < 1000]]

[None, None, None]

In [23]:
from album import album
from fileUtils import getsize
alb = album()
for ifile in files:
    print(getsize(ifile))
    #alb.getData(ifile)

125693
73320
69858
117306
66832
72812
10
80848


In [40]:
getsize('/Volumes/Music/Discog/albums/1/430501/567785.p')

10